<img src="images/Picture0.png" width=200x />

# Notebook 02 - LP Problems

### Covered in this notebook:
- Introduction to linear programming
- Creating a model in Gurobipy
    - Adding variables and constraints
    - Setting objectives
    - Adding multiple variables and constraints

### Prerequisites:
- N01
- Preliminary understanding of linear equations is useful, but not essential

### Credits:
- [Getting Started with Gurobi](https://www.youtube.com/watch?v=oBTJNRXyUu0), tutorial by [Gurobi Optimization official YouTube](https://www.youtube.com/@GurobiVideos)
- https://www.gurobi.com/resources/linear-programming-tutorial/
- https://sites.math.washington.edu/~burke/crs/515/notes/nt_1.pdf

- https://groups.google.com/g/gurobi/c/Lo_wnSlPBMQ

## Introduction

The Gurobi optimizer is a tool used to solve and analyze optimization problems.  An optimization problem maximizes or minimizes some function or set of functions within some set of feasible options.  This tool is expansive and adaptive: today, Gurobi is used by large organizations such as the NFL, the U.S. Census Bureau, GE, DoorDash, and many others in addition to being the favorite optimizer of academics and smaller professionals.

The solver is powerful!  However, it is only as effective as the designers who use it.  It is the humans behind the screen who are responsible for efficiently representing their optimization problems in mathematical terms.  In so doing, we can put this impressive machine to best use.

### Mathematical programs

A <strong>mathematical program</strong> is a formulation of an optimization problem in terms of objective functions and constraints.  In standard form, a mathematical program is expressed as the following:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\textrm{minimize or maximize the}$ $\textbf{objective function}$ $f(x_1, x_2, ..., x_n)$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\textrm{subject to the}$ $\textbf{constraint equations}$ $g_i(x_1,x_2,...,x_n)\begin{Bmatrix} \leq \\ \geq \\ = \end{Bmatrix} b_i, \forall i \in \{1,...,m\}$

The variables $x_i$ are known as <strong>decision variables</strong>: the values of these variables in the solution will <i>decide</i> how we procede in the real world.

A <strong>linear programming problem</strong> or <strong>LP problem</strong> is a mathematical program where the objective functions $f()$ and the constraint equations $g_i()$ each are linear.  This type is the most straightforward (as well as the least computationally expensive for our solver).  Let's explore LP problems as we create our first mathematical programming model with Gurobi.

## Solving LP problems with the Gurobi optimizer

Using Gurobi to solve mathematical programs will always involve the following five steps:

1. Instantiate a <strong>model</strong>
2. Add <strong>variables</strong> to the model
3. Add <strong>constraints</strong> to the model
4. Define the model's <strong>objective function</strong>
5. <strong>Solve</strong> the model

For our first foray, let's solve the following LP problem.

***

$\textbf{objective function: }$ $\textrm{minimize }$ $4x_1 + 7x_2 + 10x_3 + x_4$

$\textbf{constraints: }$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$2x_1 + 0x_2 - 3x_3 - x_4 \geq 20$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$x_1 + 2x_2 - 4x_3 + 0x_4 \geq 18$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\textrm{nonnegativity of decision variables}$

***

### Step 1: Instantiate a model

The Gurobi <strong>model</strong> object governs your program.  It holds your variables, constraints, and objective function.

The model is created by calling the model constructor `gurobipy.Model()` and storing that model as a variable.

In [3]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("LP_Problem")

Set parameter Username


### Step 2: Add variables to the model

We now call the `Model.addVar()` method on our instantiantiated model to add our <strong>decision variables</strong>.

The method takes the following arguments, none of which are required, with the following default values:

* `lb=0.0`: the <strong>lower bound</strong>
    * Note that default assumes nonnegativity.
* `ub=float('inf')`: the <strong>upper bound</strong>
    * It's good practice in larger models to set `ub` to `GRB.INFINITY` from Gurobi's constant catalogue `GRB`.  This is slightly faster for Gurobi to handle.
* `obj=0.0`, the <strong>objective coefficient</strong>
    * We'll come back to this later.
* `vtype=GRB.CONTINUOUS`: the variable <strong>type</strong>
    * Three of the five available variable types are `GRB.CONTINUOUS`, `GRB.INTEGER`, and `GRB.BINARY`.  We'll come back to the other two later.
* `name=""`: an ASCII string representing the <strong>name</strong> of your model
    * If you choose not to assign your decision variable to a Python variable, giving it a name and using `Model.getVarByName()` lets you access your variable later on.

In [4]:
## add variables to the model:

from gurobipy import GRB


### Step 3: Add constraints to the model

We now add the constraints using the `Model.addConstr()` method.  The method takes only two arguments:

* `constr`: an expression representing the <strong>constraint</strong>
* `name`: an ASCII string representing the <strong>name</strong> of the constraint

The standard Python operators (e.g. `*`, `+`, `<=`, etc.) are overloaded in Gurobi such that we can use them in expressions according to their intuitive meanings.

Here are our constraints again:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$2x_1 + 0x_2 - 3x_3 - x_4 \geq 20$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$x_1 + 2x_2 - 4x_3 + 0x_4 \geq 18$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\textrm{nonnegativity of decision variables}$

Note that our variable bounds have already taken care of our final constraint.

#### Temp elements

Variables and constraints we add to our model are stored as temp elements until the model is solved: we can use temp elements to build other temp elements, but they do not have referenceable attributes.  If we need to do something with our elements before we get our solution, we can call `Model.update()` to process pending modifications.

In [5]:
## add constraints to the model:


### Step 4: Define the model's objective function

There are two ways for us to set an <strong>objective function</strong>.  The first is to use the method `Model.setObjective()` to do so, which takes the following arguments:

* `expr`: The <strong>objective</strong> expression
* `sense`: The optimization <strong>sense</strong> (namely, `GRB.MAXIMIZE` or `GRB.MINIMIZE`)

Note that the sense is optional.  This is because the model itself has an attribute `ModelSense` which defaults to minimization, and which we can optionally override by defining `sense` here.

Alternatively, we can have Gurobi build a linear objective function from the coefficients' variables.  Here's our objective equation again:

$\textrm{minimize }$ $4x_1 + 7x_2 + 10x_3 + x_4$

Then for our problem, we could define the objective coefficients `obj` of our decision variables as we initialized them:

```
x1 = m.addVar(obj=4)
x2 = m.addVar(obj=7)
x3 = m.addVar(obj=10)
x4 = m.addVar(obj=1)
```

Gurobi treats the linear combination of our weighted decision variables as the objective function unless/until another objective is set manually.  The sense of the default objective comes from the model sense.

In [6]:
## set objective equation:


### 5. Optimize

We're now finished writing our model and are ready to optimize to our solution.

We call the method `Model.optimize()`, and Gurobi reads out our solution to the log.  (Note that the `optimize` method calls `update`, so we can now reference attributes of our model elements.)

In [7]:
## optimize model:


### Presolve

Our model is solved, and our solution is given!  Our output gives us some information about our machine power, the size of the model's matrix, the ranges of different aspects of our model.  Next, Gurobi logs information about the presolve process.

Gurobi takes a few steps in advance to reduce the complexity of our problem.  This goes beyond simply reducing the matrix; there are various tricks Gurobi can do to combine several constraints into one or to set variables' values automatically.  For example, look at our $x_3$ variable.  $x_3$ has a positive coefficient in our objective function, and we are minimizing that function.  So, we want $x_3$ to be as small as possible.  Meanwhile, $x_3$ has a negative coefficient in each of the constraint equations, and these are of the type "greater than or equal to".  So again, we want $x_3$ as small as possible.  Gurobi presolve then sets $x_3$ to its lower bound of zero right off the bat, effectively removing the variable from consideration.  Techniques like these reduce Gurobi's solving time, often by orders of magnitude in larger models.

### Retrieving our solution

The model's <strong>optimal objective</strong> attribute `objVal` gives us the overall value of our objective function $f()$, and is spit out on the last line of our log.  Of course, we probably want to know the values of our decision variables as well, so let's retrieve those.

A decision variable's value is given by the `x` or `X` attribute.  We can query this directly…

```
x1.x
```

…or we can call the `Var.getAttr()` method on the decision variable.

```
x1.getAttr('x')
```

Finally, rather than calling`getAttr` on our variables one at a time, we can call `Model.getAttr()` on the model to deliver a list containing that attribute for all the model's variables, in the order that they were added.

<i>Note: In Notebook 06, we will explore how to read and write models and their solutions to different export formats.  One of these formats (.sol) handily delivers all of our decision variables row by row with their names.  If we just want to see our solution and not work with the values it spits out in our code, this is by far the best way to get all our information at once.</i>

In [8]:
# retrieve decision variables' values in solution


## Exercise: Landscaping Business

Now it's your turn.  Use what you've learned so far to optimize the following linear programming problem:

You run a landscaping business.  You're paid by the job, and the greenery you add to the grounds affects your payment significantly.  You want to know the optimal job you should strive for.

You have the following conditions:
* You can plant trees and shrubs.  Your patrons tend to prefer at least three shrubs planted for each tree.
* Trees each take your team 1.8 hours to install, while shrub installs take 0.25 hours.  You don't want to spend more than 40 hours on greenery for a given job.
* Your deal with the nursery allows you to take out 100 credits' worth of greenery per job.  A shrub costs 1 credit and a tree costs 5 credits.
* The amount you earn per job starts at 5000 dollars, and increases by 400 dollars per tree and 40 dollars per shrub.

What is the optimal number of trees and shrubs to plant to maximize revenue?  How much is your optimal job worth?

<i>Note: While you are working, take care to remove any variables or constraints which are added in error.  Use `Model.reset()` in between revisions to clear the state of the model.</i>

In [9]:
# instantiate model

# add variables


In [10]:
# add constraints


In [11]:
# add objective function (optional)

# optimize


In [12]:
# print solution information
